In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from __future__ import print_function

import pickle
import os
import numpy as np
import tempfile
import tensorflow as tf
import matplotlib.pyplot as plt

from tf_rl.simulation import KarpathyGame
from tf_rl import simulate
from collections import OrderedDict
from euclid import Vector2

from baselines import deepq
import baselines.common.tf_util as U
from baselines.deepq.replay_buffer import ReplayBuffer
from baselines.common.schedules import LinearSchedule

In [3]:
LOG_DIR = tempfile.mkdtemp()

In [4]:
current_settings = {
    #earlier objects are eaten by later objects (pred eat prey)
    'objects': [
        'prey',
        'pred',
        'cue',
    ],
    'colors': {
        'prey': [212, 211, 208],
        'pred':  [100, 37, 0],
        'cue': [0,0,0],
    },
    'object_reward': {
        'prey': {'prey': 0.1, 'pred': -0.1, 'cue': 0.0},
        'pred': {'prey': 1.0, 'pred': -1.0, 'cue': 0.0},
    },
    "collision_penalty": 0, 
    # 0 if predators are allowed to hit each other without disappearing and reappearing at a random place
    # 1 otherwise
    'hero_bounces_off_walls': False,
    'world_size': (500,300),   
    "maximum_velocity":      {'prey': 0, 'pred': 50},
    "object_radius": 10.0,
    "cue_types": 2,
    "num_objects": OrderedDict([('prey', 5), ('pred', 5), ('cue', 1)]),
    # active means that the objects are learning
    "num_objects_active": OrderedDict([('prey', 0), ('pred', 5)]), 
    #'multiple' to create each DQN for each prey/predator
    #'one' to use one DQN for all preys/predators
    # only really matters if the preys/predators are active
    "network_prey": 'one',
    "network_pred": 'multiple',
    "num_observation_lines" : 32,
    "observation_line_length": 75.,
    "tolerable_distance_to_wall": 50,
    "wall_distance_penalty":  -1.0,
    "delta_v": 50
}

# Name to save
MODEL_NAME = 'model-5pred-2cues-500trials-80weight'
REPLAY_NAME = 'replay-5pred-2cues-500trials-80weight'
ELAPSE_NAME = 'elapse-5pred-2cues-500trials-80weight'
REWARDS_NAME = 'rewards-5pred-2cues-500trials-80weight'
COLLISIONS_NAME = 'collsions-5preds10-load-100weight-u75'

In [5]:
# create the game simulator
g = KarpathyGame(current_settings)

In [6]:
tf.reset_default_graph()

all_act = []
all_train = []
all_update = []
all_debug = []
all_replay = []

# Build graphs
if current_settings['num_objects_active']['prey'] != 0:
    if current_settings['network_prey'] == 'one':
        network_prey = 1
    else:
        network_prey = current_settings['num_objects_active']['prey']

    for i in range(network_prey):
        name = 'prey' + str(i)
        with tf.variable_scope(name):
            brain_prey = deepq.models.mlp([200, 200])
            act, train, update_target, debug = deepq.build_train(
                make_obs_ph=lambda name: U.BatchInput((g.observation_size,), name=name),
                q_func=brain_prey,
                num_actions=g.num_actions,
                optimizer=tf.train.AdamOptimizer(learning_rate=5e-4),
            )
        replay_buffer = ReplayBuffer(50000)
        all_replay.append(replay_buffer)
        all_act.append(act)
        all_train.append(train)
        all_update.append(update_target)
        all_debug.append(debug)

if current_settings['num_objects_active']['pred'] != 0:
    if current_settings['network_pred'] == 'one':
        network_pred = 1
    else:
        network_pred = current_settings['num_objects_active']['pred']

    for i in range(network_pred):
        name = 'pred' + str(i)
        with tf.variable_scope(name):
            brain_pred = deepq.models.mlp([200, 200])
            act, train, update_target, debug = deepq.build_train(
                make_obs_ph=lambda name: U.BatchInput((g.observation_size,), name=name),
                q_func=brain_pred,
                num_actions=g.num_actions,
                optimizer=tf.train.AdamOptimizer(learning_rate=5e-4),
            )
        replay_buffer = ReplayBuffer(50000)
        all_replay.append(replay_buffer)
        all_act.append(act)
        all_train.append(train)
        all_update.append(update_target)
        all_debug.append(debug)

[2017-07-28 16:09:50,141] VARIABLES collection name is deprecated, please use GLOBAL_VARIABLES instead; VARIABLES will be removed after 2017-03-02.


[2017-07-28 16:09:50,232] VARIABLES collection name is deprecated, please use GLOBAL_VARIABLES instead; VARIABLES will be removed after 2017-03-02.


[2017-07-28 16:09:50,777] VARIABLES collection name is deprecated, please use GLOBAL_VARIABLES instead; VARIABLES will be removed after 2017-03-02.


[2017-07-28 16:09:50,875] VARIABLES collection name is deprecated, please use GLOBAL_VARIABLES instead; VARIABLES will be removed after 2017-03-02.


[2017-07-28 16:09:51,314] VARIABLES collection name is deprecated, please use GLOBAL_VARIABLES instead; VARIABLES will be removed after 2017-03-02.


[2017-07-28 16:09:51,397] VARIABLES collection name is deprecated, please use GLOBAL_VARIABLES instead; VARIABLES will be removed after 2017-03-02.


[2017-07-28 16:09:51,824] VARIABLES collection name is deprecated, please use GLOBAL_VARIABLES instead; VARIABLES will be removed after 2017-03-02.


[2017-07-28 16:09:52,011] VARIABLES collection name is deprecated, please use GLOBAL_VARIABLES instead; VARIABLES will be removed after 2017-03-02.


[2017-07-28 16:09:52,446] VARIABLES collection name is deprecated, please use GLOBAL_VARIABLES instead; VARIABLES will be removed after 2017-03-02.


[2017-07-28 16:09:52,519] VARIABLES collection name is deprecated, please use GLOBAL_VARIABLES instead; VARIABLES will be removed after 2017-03-02.


In [7]:
FPS          = 30
ACTION_EVERY = 3
    
fast_mode = False
if fast_mode:
    WAIT, VISUALIZE_EVERY = False, 100
else:
    WAIT, VISUALIZE_EVERY = True, 1

elapsed = []
rewards = []
timesteps = [0]

exploration = LinearSchedule(schedule_timesteps=5000, initial_p=1.0, final_p=0.02)

# Initializing or reloading variables
# Start TensorFlow session with 2 CPUs
with U.make_session(2) as sess:
    
    # Initialize the parameters and copy them to the target network.
    U.initialize()
    for i in range(current_settings['num_objects_active']['prey']):
        name = 'prey' + str(i)
        with tf.variable_scope(name):
            update_target()
    for i in range(current_settings['num_objects_active']['pred']):
        name = 'pred' + str(i)
        with tf.variable_scope(name):
            update_target()
            
    # Run simulation
    try:
        simulate(simulation=g,
                 replay = all_replay,
                 act = all_act,
                 train = all_train,
                 update = all_update,
                 debug = all_debug,
                 fps=FPS,
                 visualize_every=VISUALIZE_EVERY,
                 action_every=ACTION_EVERY,
                 wait=WAIT,
                 disable_training=False,
                 simulation_resolution=.001,
                 save_path=None,
                 timesteps = timesteps,
                 elapsed = elapsed,
                 all_rewards = rewards,
                 percent = 1,
                 certainty = 0.0,
                 exploration = exploration,
                 draw = 1)
        
    except KeyboardInterrupt:
        print("Interrupted")
        g.shut_down_graphics()
        print('graphics shut down')
        
    # Save models    
    model_name = 'saved_graphs/' + MODEL_NAME + '-' + str(timesteps[0]) + '.ckpt'
    saver = tf.train.Saver()
    saver.save(sess, model_name)
    
    # Save replay buffers
    replay_name = 'saved_graphs/' + REPLAY_NAME + '-' + str(timesteps[0]) + '.pkl'
    with open(replay_name, "wb") as f:
        pickle.dump(all_replay, f)
        
    # Save trial times
    elapse_name = 'saved_graphs/' + ELAPSE_NAME + '-' + str(timesteps[0]) + '.pkl'
    with open(elapse_name, "wb") as f:
        pickle.dump(elapsed, f)
        
    # Save rewards
    rewards_name = 'saved_graphs/' + REWARDS_NAME + '-' + str(timesteps[0]) + '.pkl'
    with open(rewards_name, "wb") as f:
        pickle.dump(rewards, f)
        
    # Save collisions
    collisions_name = 'saved_graphs/' + COLLISIONS_NAME + '-' + str(timesteps[0]) + '.pkl'
    with open(collisions_name, "wb") as f:
        pickle.dump(g.collisions, f)

0
Interrupted
graphics shut down


In [8]:
# FOR DEBUGGING
# Visualize graph in TensorBoard
tf.summary.FileWriter(LOG_DIR, tf.get_default_graph())

# Get all current variables
# tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)